# Regmap Parsing

In [2]:
import clock_models
import numpy as np

In [3]:
clk104 = clock_models.CLK104("clockfile.txt")

In [4]:
clk104.PLL2_CLK = 2457.6 # Main Reference Clock
# clk104.PLL2_CLK = 3000 # Main Reference Clock

In [5]:
clk104.SYSREF_CLK = 7.69

In [6]:
clk104.get_register_dump()

[(0, 16),
 (2, 512),
 (3, 774),
 (4, 1232),
 (5, 1371),
 (6, 1536),
 (12, 3153),
 (13, 3332),
 (256, 65546),
 (257, 65877),
 (258, 66133),
 (259, 66304),
 (260, 66594),
 (261, 66816),
 (262, 67312),
 (263, 67377),
 (264, 67594),
 (265, 67925),
 (266, 68181),
 (267, 68352),
 (268, 68642),
 (269, 68864),
 (270, 69360),
 (271, 69424),
 (272, 69642),
 (273, 69973),
 (274, 70229),
 (275, 70400),
 (276, 70690),
 (277, 70912),
 (278, 71408),
 (279, 71475),
 (280, 71690),
 (281, 72021),
 (282, 72277),
 (283, 72448),
 (284, 72738),
 (285, 72960),
 (286, 73457),
 (287, 73488),
 (288, 73736),
 (289, 74069),
 (290, 74325),
 (291, 74496),
 (292, 74786),
 (293, 75008),
 (294, 75504),
 (295, 75569),
 (296, 75786),
 (297, 76117),
 (298, 76373),
 (299, 76544),
 (300, 76834),
 (301, 77056),
 (302, 77552),
 (303, 77616),
 (304, 77834),
 (305, 78165),
 (306, 78421),
 (307, 78592),
 (308, 78882),
 (309, 79104),
 (310, 79601),
 (311, 79616),
 (312, 79872),
 (313, 80131),
 (314, 80385),
 (315, 80704),
 (316,

In [48]:
clk104.lmk.update(printDebug=True)

SEL_MODE: CLK_IN_1_MANUAL
Input divider: 8
PLL1 Phase Detector Frequency: 1.25
PLL1N Divider: 128
Expected VCXO Frequency: 160.0
PLL2 Input Frequency: 6.4
PLL2_P: 2
PLL2_N: 192
PLL2 Output Frequency: 2457.6
SYSREF DIVIDER: 320
SYSREF FREQ: 7.68
Output Branch  0 DIV: 10
Output Branch  0 CLK_PD: ENABLED
Output Branch  0 SDCLK_PD: ENABLED
Output Branch  0 DCLK_FMT: LVDS
Output Branch  0 SDCLK_FMT: HSDS_8_MA
Output Branch  0 DCLK_POL: NORMAL
Output Branch  0 SDCLK_POL: NORMAL
Output Branch  0 SDCLK_MUX: SYSREF_OUTPUT
Output Branch  0 DCLK_MUX: DIVIDER_ONLY
Output Branch  0 DCLK_FREQ: 245.76
Output Branch  0 SDCLK_FREQ: 7.68
Output Branch  0 DCLK_ACTIVE: True
Output Branch  0 SDCLK_ACTIVE: True
Output Branch  2 DIV: 10
Output Branch  2 CLK_PD: ENABLED
Output Branch  2 SDCLK_PD: ENABLED
Output Branch  2 DCLK_FMT: POWERDOWN
Output Branch  2 SDCLK_FMT: HSDS_8_MA
Output Branch  2 DCLK_POL: NORMAL
Output Branch  2 SDCLK_POL: NORMAL
Output Branch  2 SDCLK_MUX: SYSREF_OUTPUT
Output Branch  2 DCLK_

In [6]:
clk104.lmk.write_register_dump("clockfile_out.txt")

In [7]:
!diff clockfile.txt clockfile_out.txt

136c136
< R8191	0x1FFF53
\ No newline at end of file
---
> R8191	0x1FFF53


In [8]:
def parse_register_file(filename):
    with open(filename, "r") as f:
        lines = f.read().strip().split("\n")

    ret = []

    for line in lines:
        a,b = line.split("\t")
        data = int(b, 16)

        ret.append((0x1fff & (data >> 8), data & 0xFF))

    return ret

In [9]:
prog_file = parse_register_file("clockfile.txt")

In [10]:
print([a[0] for a in prog_file])

[0, 0, 2, 3, 4, 5, 6, 12, 13, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 369, 370, 380, 381, 358, 359, 360, 361, 362, 363, 364, 365, 366, 371, 386, 387, 388, 389, 392, 393, 394, 395, 8189, 8190, 8191]


In [11]:
if 0:
    for addr,data in prog_file:
        # print(f"Explaining register 0x{addr:x} = 0x{data:x}")
        if not addr in register_by_addr:
            print(f"Unhandled register: {hex(addr)}, skipping ...")
            print()
            continue

        register = register_by_addr[addr]
        print(hex(addr))
        register.parse(data)

        for field in register.fields:
            val = field.get()
            if field.name == "CONST":
                continue

            print(f"    {field.name}")
            print(f"        Description: {field.description}")
            print(f"        Value:       {val}")
            if field.valid_type == "enum":
                print(f"        ValDesc:     {field.value_description}")
            print()